## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Ridgefield,40.8343,-74.0088,53.92,84,90,5.75,US,drizzle,2021-11-22 02:46:33
1,1,Punta Arenas,-53.1500,-70.9167,39.31,75,75,25.32,CL,broken clouds,2021-11-22 02:51:11
2,2,Upernavik,72.7868,-56.1549,4.35,87,40,6.62,GL,scattered clouds,2021-11-22 02:51:12
3,3,Fairbanks,64.8378,-147.7164,-10.39,75,1,0.00,US,clear sky,2021-11-22 02:48:50
4,4,Atuona,-9.8000,-139.0333,78.26,74,100,10.78,PF,overcast clouds,2021-11-22 02:51:12


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
4,4,Atuona,-9.8000,-139.0333,78.26,74,100,10.78,PF,overcast clouds,2021-11-22 02:51:12
6,6,Kapaa,22.0752,-159.3190,82.38,71,40,3.00,US,scattered clouds,2021-11-22 02:51:13
12,12,Poum,-20.2333,164.0167,78.71,66,3,5.84,NC,clear sky,2021-11-22 02:51:15
16,16,Vaini,-21.2000,-175.2000,80.76,94,75,5.75,TO,broken clouds,2021-11-22 02:51:18
18,18,Kanker,20.2719,81.4931,78.55,76,81,0.96,IN,broken clouds,2021-11-22 02:51:18
19,19,Progreso,21.2833,-89.6667,77.11,82,2,11.74,MX,clear sky,2021-11-22 02:51:19
25,25,Port Hedland,-20.3167,118.5667,87.85,45,100,11.50,AU,overcast clouds,2021-11-22 02:51:22
27,27,Avarua,-21.2078,-159.7750,80.65,74,90,4.61,CK,overcast clouds,2021-11-22 02:51:23
28,28,Sinnamary,5.3833,-52.9500,78.40,87,9,7.90,GF,clear sky,2021-11-22 02:51:23
35,35,Hambantota,6.1241,81.1185,77.34,88,0,4.61,LK,clear sky,2021-11-22 02:51:26


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                169
City                   169
Lat                    169
Lng                    169
Max Temp               169
Humidity               169
Cloudiness             169
Wind Speed             169
Country                169
Current Description    169
Date                   169
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
4,4,Atuona,-9.8000,-139.0333,78.26,74,100,10.78,PF,overcast clouds,2021-11-22 02:51:12
6,6,Kapaa,22.0752,-159.3190,82.38,71,40,3.00,US,scattered clouds,2021-11-22 02:51:13
12,12,Poum,-20.2333,164.0167,78.71,66,3,5.84,NC,clear sky,2021-11-22 02:51:15
16,16,Vaini,-21.2000,-175.2000,80.76,94,75,5.75,TO,broken clouds,2021-11-22 02:51:18
18,18,Kanker,20.2719,81.4931,78.55,76,81,0.96,IN,broken clouds,2021-11-22 02:51:18
...,...,...,...,...,...,...,...,...,...,...,...
675,675,Kudahuvadhoo,2.6708,72.8944,82.06,76,77,14.54,MV,broken clouds,2021-11-22 03:10:18
676,676,Corowa,-36.0333,146.3833,79.41,44,2,2.62,AU,clear sky,2021-11-22 03:10:19
677,677,Kavaratti,10.5669,72.6420,80.60,74,13,3.83,IN,few clouds,2021-11-22 03:10:19
685,685,San Francisco Del Mar,16.2333,-94.6500,79.20,74,1,22.21,MX,clear sky,2021-11-22 03:10:23


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.26,overcast clouds,-9.8000,-139.0333,
6,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
12,Poum,NC,78.71,clear sky,-20.2333,164.0167,
16,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
18,Kanker,IN,78.55,broken clouds,20.2719,81.4931,
19,Progreso,MX,77.11,clear sky,21.2833,-89.6667,
25,Port Hedland,AU,87.85,overcast clouds,-20.3167,118.5667,
27,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,
28,Sinnamary,GF,78.40,clear sky,5.3833,-52.9500,
35,Hambantota,LK,77.34,clear sky,6.1241,81.1185,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.notnull().sum()

City                   169
Country                169
Max Temp               169
Current Description    169
Lat                    169
Lng                    169
Hotel Name             169
dtype: int64

In [12]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.26,overcast clouds,-9.8000,-139.0333,Villa Enata
6,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Poum,NC,78.71,clear sky,-20.2333,164.0167,Gîte kajeon
16,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Kanker,IN,78.55,broken clouds,20.2719,81.4931,STARiHOTEL Kanker
19,Progreso,MX,77.11,clear sky,21.2833,-89.6667,Playa Linda Hotel
25,Port Hedland,AU,87.85,overcast clouds,-20.3167,118.5667,The Esplanade Hotel
27,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,Paradise Inn
28,Sinnamary,GF,78.40,clear sky,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
35,Hambantota,LK,77.34,clear sky,6.1241,81.1185,Bungalow 63


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))